In [1]:
from typing import Tuple

import itertools

import numpy
import pandas

from matplotlib import pyplot as plt

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-hackathon23_somnolence/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
country = 'france'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')
df_covid_measures = pandas.read_csv(
    filepath_or_buffer = f'../../../data/intermediate/{country}/government_covid_measures.csv'
)

In [3]:
df_covid_workplace_lockdowns = df_covid_measures[['date', 'workplace_closing']]
df_covid_workplace_lockdowns.to_csv('covid_workplace_lockdowns.csv', index = False)

In [4]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)

    # manually created from df_covid_workplace_lockdowns
    df_workplace_lockdowns = pandas.DataFrame([
        {'holiday': 'lockdown_1', 'ds': '2020-06-22', 'lower_window': 0, 'ds_upper': '2020-08-13'},
        {'holiday': 'lockdown_2', 'ds': '2022-03-14', 'lower_window': 0, 'ds_upper': '2022-06-19'},
        {'holiday': 'lockdown_2', 'ds': '2020-05-11', 'lower_window': 0, 'ds_upper': '2020-06-21'},
        {'holiday': 'lockdown_2', 'ds': '2020-08-14', 'lower_window': 0, 'ds_upper': '2020-10-29'},
        {'holiday': 'lockdown_2', 'ds': '2020-11-28', 'lower_window': 0, 'ds_upper': '2021-02-25'},
        {'holiday': 'lockdown_2', 'ds': '2021-03-02', 'lower_window': 0, 'ds_upper': '2021-03-04'},
        {'holiday': 'lockdown_2', 'ds': '2021-04-03', 'lower_window': 0, 'ds_upper': '2021-05-02'},
        {'holiday': 'lockdown_2', 'ds': '2021-05-19', 'lower_window': 0, 'ds_upper': '2022-03-13'},
        {'holiday': 'lockdown_3', 'ds': '2020-03-17', 'lower_window': 0, 'ds_upper': '2020-05-10'},
        {'holiday': 'lockdown_3', 'ds': '2020-10-30', 'lower_window': 0, 'ds_upper': '2020-11-27'},
        {'holiday': 'lockdown_3', 'ds': '2021-02-26', 'lower_window': 0, 'ds_upper': '2021-03-01'},
        {'holiday': 'lockdown_3', 'ds': '2021-03-05', 'lower_window': 0, 'ds_upper': '2021-04-02'},
        {'holiday': 'lockdown_3', 'ds': '2021-05-03', 'lower_window': 0, 'ds_upper': '2021-05-18'},
    ])
    for t_col in ['ds', 'ds_upper']:
        df_workplace_lockdowns[t_col] = pandas.to_datetime(df_workplace_lockdowns[t_col], format = '%Y-%m-%d')
    df_workplace_lockdowns['upper_window'] = (df_workplace_lockdowns['ds_upper'] - df_workplace_lockdowns['ds']).dt.days
    df_workplace_lockdowns = df_workplace_lockdowns.sort_values(by = 'ds', axis = 'index', ascending = True, inplace= False)

    return df_mrd, df_workplace_lockdowns

In [5]:
df_mrd, df_workplace_lockdowns = make_dataset(df)

In [6]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [7]:
df_train, df_test = train_test_split(df_mrd, 12)

In [8]:
def train_model(df: pandas.DataFrame, df_holidays: pandas.DataFrame, param_grid: dict, eval_metric: str = 'rmse', eval_metric_optimize: str = 'min') -> Prophet:
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    eval_metric_values = []  # Store the eval_metric_values for each params here

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(holidays=df_holidays, **params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, horizon = '60 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1, monthly=True)
        eval_metric_values.append(df_p[eval_metric].values[0])

    # Find the best parameters
    tuning_results = pandas.DataFrame(all_params)
    tuning_results[eval_metric] = eval_metric_values

    tuning_results.sort_values(by = eval_metric, axis = 'index', ascending = True, inplace = True)
    best_params = {}
    if eval_metric_optimize == 'min':
        best_params = tuning_results.iloc[0].to_dict()
    elif eval_metric_optimize == 'max':
        best_params = tuning_results.iloc[-1].to_dict()
    else:
        raise ValueError("Invalid Parameter Value: param 'eval_metric_optimize' may only have values 'min' or 'max'.")


    optimal_model = Prophet(holidays=df_holidays, **params).fit(df)
    return optimal_model

In [9]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

model = train_model(df_train, df_workplace_lockdowns, param_grid)

12:12:56 - cmdstanpy - INFO - Chain [1] start processing
12:12:56 - cmdstanpy - INFO - Chain [1] done processing
12:12:56 - cmdstanpy - ERROR - Chain [1] error: error during processing Communication error on send
Optimization terminated abnormally. Falling back to Newton.
12:12:56 - cmdstanpy - INFO - Chain [1] start processing
12:13:55 - cmdstanpy - INFO - Chain [1] done processing
12:13:56 - cmdstanpy - INFO - Chain [1] start processing
12:13:56 - cmdstanpy - INFO - Chain [1] start processing
12:13:56 - cmdstanpy - INFO - Chain [1] start processing
12:13:56 - cmdstanpy - INFO - Chain [1] start processing
12:13:56 - cmdstanpy - INFO - Chain [1] start processing
12:13:56 - cmdstanpy - INFO - Chain [1] start processing
12:13:56 - cmdstanpy - INFO - Chain [1] start processing
12:13:56 - cmdstanpy - INFO - Chain [1] start processing
12:14:09 - cmdstanpy - INFO - Chain [1] done processing
12:14:10 - cmdstanpy - INFO - Chain [1] start processing
12:14:10 - cmdstanpy - INFO - Chain [1] done 

KeyboardInterrupt: 

In [ ]:
def test_model(df_test: pandas.DataFrame, model: Prophet) -> Tuple[pandas.DataFrame, float]:
    "return predicted values and rmse"
    df_predicted: pandas.DataFrame = model.predict(df_test)
    rmse: float = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [ ]:
df_predicted, rmse = test_model(df_test, model)

In [ ]:
df_predicted

In [ ]:
rmse

In [ ]:
with open('france_prophet_lockdowns_optimized_model.json', 'w') as f:
    f.write(model_to_json(model))

In [ ]:
df_predicted

In [ ]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01']})

In [ ]:
df_future_prediction: pandas.DataFrame = model.predict(df_future)

In [ ]:
df_future_prediction